In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pymysql as pymy

In [ ]:
from env import host, username, password


In [ ]:
def get_db_url(db, username=username, host=host, password=password):
    return f'mysql+pymysql://{username}:{password}@{host}/{db}'

In [ ]:
#4. #b. Use your function to obtain a connection to the employees database.

sql_query = 'SELECT * FROM employees'


In [ ]:
employees = pd.read_sql(sql_query, env4.get_db_url('employees'))


In [ ]:
employees.to_csv('employees.csv')


In [ ]:
employees = pd.read_csv('employees.csv', index_col=0)
employees.head()

#5. a. Intentionally make a typo in the database url. What kind of error message do you see?

You get this: "ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '' at line 1")"


In [ ]:
# 6. Read the employees and titles tables into two separate DataFrames.

# sql_query = 'SELECT * FORM titles'

# titles = pd.read_sql(sql_query, get_db_url('employees'))
# titles.head()

In [ ]:
#d. Read the employees and titles tables into two separate dataframes
sql_query = 'SELECT * FROM titles'
titles = pd.read_sql(sql_query, get_db_url('employees'))
titles.head()

In [ ]:
title_query = 'select * from titles;'
titles = pd.read_sql(title_query, env4.get_db_url('employees'))
titles.head()

In [ ]:
#Merging tables:
#all_emp_titles = employees.merge(titles, on='emp_no')
#all_emp_titles.head()#

In [ ]:
#How many rows and columns do you have in each DataFrame? Is that what you expected?

_employees shape: : (300024, 6)

title shape: (443308, 4)

#Display the summary statistics for each DataFrame.
titles.describe()
employees.describe()

#How many unique titles are in the titles DataFrame?
titles.unique()
employees.unique()

#What is the oldest date in the to_date column?
titles.to_date.min()
#What is the most recent date in the to_date column?#
titles.to_date.max()

In [ ]:
#Exercise 2

In [ ]:
# Copy the users and roles DataFrames from the examples above.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

In [ ]:
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

In [ ]:
# 2. What is the result of using a right join on the DataFrames?

right_join = pd.merge(users, 
                    roles, 
                    left_on='role_id', 
                    right_on='id', 
                    how='right').drop(columns='role_id').rename(columns={'id_x': 'id', 
                                                                         'name_x': 'employee',
                                                                         'id_y': 'role_id',
                                                                         'name_y': 'role'}
                                                                )
right_join

In [ ]:
#. What is the result of using an outer join on the DataFrames?

outer_join = pd.merge(users, 
         roles, 
         left_on='role_id', 
         right_on='id', 
         how='outer').drop(columns='role_id').rename(columns={'id_x': 'id', 
                                                            'name_x': 'employee',
                                                            'id_y': 'role_id',
                                                            'name_y': 'role'}
                                                    )
outer_join

In [ ]:
# 4. What happens if you drop the foreign keys from the DataFrames and try to merge them?

users_drop = users.drop(columns='role_id')
users_drop

#When the instructions say to load a dataset, you can pass the name of the dataset as a string to the data function to load the dataset. You can also view the documentation for the data set by passing the show_doc keyword argument.

#mpg = data('mpg') # load the dataset and store it in a variable
#### data('mpg', show_doc=True) # view the documentation for the dataset

In [ ]:
# 5. Load the mpg dataset from PyDataset. 
# 6. Output and read the documentation for the mpg dataset.

data('mpg', show_doc=True)

In [ ]:
mpg = data('mpg')
mpg.head()

In [ ]:
#How many rows and columns are in the dataset?

#Check out your column names and perform any cleanup you may want on them.

#Display the summary statistics for the dataset.

#How many different manufacturers are there? 15
mpg.manufacturer.nunique()


#How many different models are there?  38
mpg.model.nunique()


In [ ]:
#. Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.

mpg['average_mileage'] = (mpg.cty + mpg.hwy) / 2


In [ ]:
# 12 .Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.

mpg['mileage_differenc'] = mpg['cty'].subtract(mpg['hwy'])
#mpg['mileage_differenc'] = mpg["cty"] - mpg["hwy"]

In [ ]:
# 14. Create a new column on the mpg dataset named is_automatic that holds boolean values denoting 
#whether the car has an automatic transmission.

mpg['is_automatic'] = np.where(mpg.trans.str.startswith('a'), 'auto', 'manual')


In [ ]:
# 15. Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?

#mpg.groupby('manufacturer').average_mileage.mean().sort_values(ascending=False).idxmax()

In [ ]:
# 16. Do automatic or manual cars have better miles per gallon?

mpg.groupby('is_automatic')[['cty', 'hwy']].mean()


In [ ]:
#Exercises 3

In [ ]:
chipotle_sql_query = '''
                     SELECT *
                     FROM orders;
                     '''

In [ ]:
orders = pd.read_sql(chipotle_sql_query, env4.get_db_url('chipotle'))
orders.head()

In [ ]:
#orders.shape

#orders.info()

In [ ]:
# 2. What is the total price for each order?

orders['item_price'] = orders.item_price.str.replace('$', '').astype(float)
orders.info()

In [ ]:
order_totals = orders.groupby('order_id')['item_price'].sum()


In [ ]:
# What are the most popular 3 items?

top_three = orders.groupby('item_name')['quantity'].sum().nlargest(3)
# Alt method: orders.groupby('item_name')['quantity'].sum().sort_values(ascending=False)[:3]
top_three

In [ ]:
# 4. Which item has produced the most revenue?

# Alt method: orders.groupby('item_name').item_price.sum().sort_values(ascending=False).head(1)

orders.groupby('item_name')['item_price'].sum().nlargest(1)

In [ ]:
# 5. Join the employees and titles DataFrames together.

#Alt method: emp_titles = employees.merge(titles, on='emp_no')

emp_titles = pd.merge(employees, titles, on='emp_no')
emp_titles.head()

In [ ]:
# 6. For each title, find the hire date of the employee that was hired most recently with that title.
#Alt method: emp_titles.groupby('title').agg(max)[['hire_date']]

emp_titles.groupby('title').agg({'hire_date': 'max'})

In [ ]:
# 7. Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)

dept_title_query = '''

                    SELECT t.emp_no, 
                    t.title, 
                    t.from_date, 
                    t.to_date, 
                    d.dept_name 
                    FROM departments AS d 
                    JOIN dept_emp AS de USING(dept_no) 
                    JOIN titles AS t USING(emp_no);
                    
                    '''


In [ ]:
dept_titles = pd.read_sql(dept_title_query, env4.get_db_url('employees'))

#dept_titles = employees.to_csv('dept_titles.csv')
#pd.read_csv(dept_titles, index_col = 0)

In [ ]:
current_titles = dept_titles[dept_titles.to_date == dept_titles.to_date.max()]


dept_titles.to_date == dept_titles.to_date.max() compares each date in the 'to_date' column with the maximum date. 

It checks if the 'to_date' of each record is equal to the maximum date. 

This comparison will result in a series of True and False values.

In [ ]:
#Alt method: current_titles = dept_titles.query('to_date == to_date.max()')


In [ ]:
current_titles_crosstab = pd.crosstab(current_titles.dept_name, current_titles.title)
current_titles_crosstab